<a href="https://colab.research.google.com/github/ching011500/sofa_potato/blob/main/Pycaret_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install Pycaret

In [ ]:
!pip install pycaret

## load sample dataset

In [ ]:
from pycaret.datasets import get_data
data = get_data('diabetes')

 ## Functional API（函式式寫法）

In [ ]:
from pycaret.classification import *
s = setup(data, target = 'Class variable', session_id = 123)

## 📊 PyCaret `setup()` 回傳的設定摘要解釋

這是 PyCaret 在執行 `setup()` 之後顯示的設定表，幫助我們快速了解資料前處理和建模流程設定如下：

| 編號 | 說明 (Description) | 值 (Value) | 詳細解釋 |
|------|----------------------|-------------|-----------|
| 0 | Session id | 123 | 隨機種子，用來確保結果「可重現」。每次使用同樣的數字，交叉驗證分法都一樣。 |
| 1 | Target | Class variable | 你指定的目標變數名稱（要預測的欄位）。 |
| 2 | Target type | Binary | 類型為二元分類（Binary Classification），例如 0/1、是/否。 |
| 3 | Original data shape | (768, 9) | 原始資料共有 768 筆資料，9 個欄位（包含目標變數）。 |
| 4 | Transformed data shape | (768, 9) | 前處理後的資料維度。這裡大小沒變，表示資料已經很乾淨。 |
| 5 | Transformed train set shape | (537, 9) | 訓練資料筆數（70%）：537 筆資料。 |
| 6 | Transformed test set shape | (231, 9) | 測試資料筆數（30%）：231 筆資料。 |
| 7 | Numeric features | 8 | 數值型欄位有 8 個（剩下 1 個是目標變數）。 |
| 8 | Preprocess | True | PyCaret 有啟用前處理（補缺值、標準化、編碼等）。 |
| 9 | Imputation type | simple | 缺失值補值策略：使用簡單補法（Simple Imputation）。 |
| 10 | Numeric imputation | mean | 數值型欄位缺失值補平均值。 |
| 11 | Categorical imputation | mode | 類別欄位缺失值補眾數（最常出現的類別）。 |
| 12 | Fold Generator | StratifiedKFold | 使用 Stratified K-Fold（分層交叉驗證）。可保持類別比例一致。 |
| 13 | Fold Number | 10 | 使用 10-fold Cross Validation。 |
| 14 | CPU Jobs | -1 | 使用所有 CPU 核心加速訓練。 |
| 15 | Use GPU | False | 沒使用 GPU。如果模型支援 GPU，可以手動開啟。 |
| 16 | Log Experiment | False | 沒有使用實驗記錄功能（如 MLflow）。 |
| 17 | Experiment Name | clf-default-name | 預設的實驗名稱（可自訂）。 |
| 18 | USI | 238b | 唯一識別碼（Unique Session Identifier），用於追蹤實驗。 |

---

📌 **小提醒**：這個表只會在 `setup()` 成功執行時出現，幫助我們確認資料預處理與建模環境已成功建立。

## Compare Models

In [ ]:
# functional API
best = compare_models()
print(best)

## Analyze Model

In [ ]:
# functional API
evaluate_model(best)

## 📊 PyCaret 模型分析報告：Logistic Regression（重點圖表）

### 🔄 Pipeline Plot
這張圖呈現了 PyCaret 自動建立的完整資料處理流程（Pipeline）：

Raw Data → SimpleImputer（數值） → SimpleImputer（類別） → CleanColumnNames → LogisticRegression


- **SimpleImputer**：補齊遺失值，數值補平均、類別補眾數
- **CleanColumnNames**：欄位名稱標準化（移除空格、特殊字元等）
- **LogisticRegression**：使用邏輯斯迴歸作為分類模型

---

### ⚙️ Hyperparameters（模型參數）
使用的 Logistic Regression 預設超參數如下：

| 參數 | 說明 |
|------|------|
| C = 1.0 | 正則化強度（越小代表越強正則化） |
| penalty = l2 | 使用 L2 正則化 |
| solver = lbfgs | 使用牛頓法優化器 |
| max_iter = 1000 | 最多迭代次數 |
| fit_intercept = True | 是否估計截距項 |
| random_state = 123 | 設定隨機種子以利重現 |

---

### 📈 AUC（ROC Curve）分析

本圖呈現 Logistic Regression 模型在不同判別閾值（threshold）下的分類能力。

共包含四條 ROC 曲線，各自意義如下：

- **ROC of class 0（AUC = 0.86）**  
  表示以 class 0 為 positive 類別時，模型將 class 0 判別正確的能力（即：真陽性率 vs 假陽性率）。

- **ROC of class 1（AUC = 0.86）**  
  表示以 class 1 為 positive 類別時，模型將 class 1 判別正確的能力。

- **micro-average ROC（AUC = 0.86）**  
  是基於所有樣本的 TP、FP、FN、TN 加總後計算出來的平均 AUC。若樣本數不平衡時，會偏向樣本數較多的類別。

- **macro-average ROC（AUC = 0.86）**  
  是對每一個類別各自計算 AUC，再進行平均，**不考慮樣本數比例**，強調各類別公平的表現。

📌 **結論**：  
四條 AUC 值皆為 0.86，代表模型對兩類別（class 0 和 class 1）皆有良好的區辨能力，且整體表現穩定、無偏頗。

---

### 📊 Confusion Matrix（混淆矩陣）

|               | 預測 0 | 預測 1 |
|---------------|--------|--------|
| **實際 0**     | 132    | 18     |
| **實際 1**     | 38     | 43     |

- **True Negatives (TN) = 132**  
  實際為 0，模型也正確預測為 0 → ✅ 沒發生事件，成功辨識

- **False Positives (FP) = 18**  
  實際為 0，但模型預測為 1 → ❌ 錯判為發生事件（假警報）

- **False Negatives (FN) = 38**  
  實際為 1，但模型預測為 0 → ❌ 該預測會發生但漏判（錯失）

- **True Positives (TP) = 43**  
  實際為 1，模型也預測為 1 → ✅ 有事件發生，成功抓到

#### 📌 總結與模型意涵：

- 模型對 **class 0**（沒事）預測能力強，因為 TN = 132，FP 只有 18
- 模型對 **class 1** 的表現稍弱，因為漏判了 38 筆（FN），成功抓到的 TP 只有 43 筆

你也可以搭配這些計算：

- **Accuracy** = (TP + TN) / 所有資料 = (132 + 43) / 231 ≈ 0.758
- **Precision (class 1)** = TP / (TP + FP) = 43 / (43 + 18) ≈ 0.705
- **Recall (class 1)** = TP / (TP + FN) = 43 / (43 + 38) ≈ 0.531
- **F1 Score** = 調和平均 ≈ 0.606
這些數字也會出現在 Class Report 中。

---

### 🧭 Threshold Plot（分類門檻分析圖）

此圖用來觀察 **不同閾值（threshold）對 precision、recall、F1 score 的影響**。
透過此圖，我們能找出**最佳的分類門檻**。

---

#### 🔍 三條主要曲線說明：

| 曲線代表 | 指標說明 |
|----------|-----------|
| 藍線     | **Precision（精確率）**：預測為 1 中，有多少是真的 |
| 綠線     | **Recall（召回率）**：實際為 1 中，有多少被找出來 |
| 紅線     | **F1 Score**：精確率與召回率的調和平均（綜合指標） |

- **最佳 threshold ≈ 0.48**（F1 分數最高點）

可以根據這個圖選擇最佳 threshold，再進行預測：

```python
predict_model(model, probability_threshold=0.48)
```

---

### 🧪 Class Report（分類報告）

| 類別 | Precision | Recall | F1-score | Support |
|------|-----------|--------|----------|---------|
| 1    | 0.705     | 0.531  | 0.606    | 81      |
| 0    | 0.776     | 0.880  | 0.825    | 150     |

- 類別 0 較易正確分類，類別 1 表現略弱
- 顯示模型在處理正負樣本上的不平衡狀況

---


## Predictions

In [ ]:
# functional API
predict_model(best) # test_data

In [ ]:
# functional API
print(data.shape)  # Print the shape (rows, columns) of the dataset
predictions = predict_model(best, data=data)  # Use the best trained model to make predictions on the full data
predictions.head()

In [ ]:
# functional API
predictions = predict_model(best, data=data, raw_score=True) # return raw class probabilities
predictions.head()

## Save the model


In [ ]:
# functional API
save_model(best, 'best_model')

## To load the model back in environment

In [ ]:
# functional API
loaded_model = load_model('best_model')
print(loaded_model)